In [ ]:
import torch

from load_checkpoint import load_checkpoint
from hparams import get_hparams_from_file
from inference import SynthesizerInf
from text.symbols import symbols
from text.cleaner import japanese_cleaner
from craft_vits import get_text
from torch.utils.mobile_optimizer import optimize_for_mobile

In [ ]:
hps = get_hparams_from_file("./configs/ljs_base.json")

net_g = SynthesizerInf(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = load_checkpoint("./logs/ljs_base/G_119000.pth", net_g, None)
net_g = net_g.cpu()

In [ ]:
torch.set_grad_enabled(False)
longtext = "どうしてこうなるんだろう。初めて、好きな人が出来た。一生ものの友だちができた。嬉しいことが二つ重なって。その二つの嬉しさが、また、たくさんの嬉しさを連れてきてくれて。夢のように幸せな時間を手に入れたはずなのに…なのに、どうして、こうなっちゃうんだろう。　どうしてこうなるんだろう。初めて、好きな人が出来た。一生ものの友だちができた。嬉しいことが二つ重なって。その二つの嬉しさが、また、たくさんの嬉しさを連れてきてくれて。夢のように幸せな時間を手に入れたはずなのに…なのに、どうして、こうなっちゃうんだろう。 えっとね……。管理人の司令官でしょ？　あとは……　レッドバロン、夜の貴公子読むだけでも恥ずかしいな。　それにエデンの女王と、ハリーって人たちが常連。　えっとね……。管理人の司令官でしょ？　あとは……　レッドバロン、夜の貴公子読むだけでも恥ずかしいな。　それにエデンの女王と、ハリーって人たちが常連。　このエデンの女王？　もさ、なんか怪しいって　思っただけで、みゃーこ先輩みたいにアニメには　ないこと書いてるってわかんなかったし。　ちゅっ、ちゅっ、じゅるるっ、全部、んぁ、んっ、　口に、んんっ、全部、飲む、からぁっ、　お兄ちゃんの、ちょうだい、ちょうだぁい……っ。　んくっ、んっ、じゅるっ、んん、んっ、ちゅっ、ちゅるっ、　んん、ちゅるっ、じゅるるるっ、じゅるっ、　ん、ん、んっ、んんッ！"

stn_tst = get_text(japanese_cleaner(longtext), hps)
x = stn_tst.unsqueeze(0)
x_lengths = torch.LongTensor([stn_tst.size(0)])

sm = torch.jit.trace(net_g, (x, x_lengths))
sm = optimize_for_mobile(sm)

In [ ]:
sm.save("net_g.pt")